# Additional parsing of the dataset

Here, we created the script where we additionally parsed data and combined training13b.json and retrived_articles_sampled.json file
We removed data which may not be relevant for eaxtraction

In [32]:
import json
import random


# retreived_articles_sampled.json path
#retreived_articles_sampled_path = "../datasets/training/retrieved_articles_sampled.json"
#retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_1/retrieved_articles_sampled_test_batch_1.json"
#retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_2/retreived_articles_sampled_test_batch_2.json"
retreived_articles_sampled_path = "../datasets/test_golden_answers/batch_3/retreived_articles_sampled_test_batch_3.json"

# path to the original file which is downloaded form the BioASQ site path
#original_downloaded_file_path = '../datasets/training/training13b.json'
#original_downloaded_file_path = '../datasets/test_golden_answers/batch_1/BioASQ-task13bPhaseB-testset1'
#original_downloaded_file_path = '../datasets/test_golden_answers/batch_2/BioASQ-task13bPhaseB-testset2'
original_downloaded_file_path = '../datasets/test_golden_answers/batch_3/BioASQ-task13bPhaseB-testset3'

# link to parsed json file
#parsed_data_path = "../datasets/training/parsed_data_final.json"
#parsed_data_path = "../datasets/test_golden_answers/batch_1/parsed_data_final_test_batch_1.json"
#parsed_data_path = "../datasets/test_golden_answers/batch_2/parsed_data_final_test_batch_2.json"
parsed_data_path = "../datasets/test_golden_answers/batch_3/parsed_data_final_test_batch_3.json"

In [33]:
# Load sample document dataset
with open(retreived_articles_sampled_path, 'r', encoding='utf-8') as f:
    retreived_articles_sampled_data = json.load(f)['results']  # assumes your sample is under the 'results' key

# Load BioASQ training dataset
with open(original_downloaded_file_path, 'r', encoding='utf-8') as f:
    training13b_data = json.load(f)['questions']  

In [34]:
# this is function that converts the PubMed ID to a link, as same as showed in the training13b.json
def convert_pid_to_link(pid):
    """
    Convert PubMed ID to link.
    """
    link_prefix = "http://www.ncbi.nlm.nih.gov/pubmed/"
    return link_prefix + pid if pid and link_prefix not in pid else pid

Now, we will extract relevant info from each dataset
retreived_articles_sampled.json -> [question id, question, error_rate, ground truth list, articles (with info)]
training_articles_sampled.json -> [question id, ground truth snippets]

We will first create dictionary from training13b.json, because we need only groun truth snippets


In [35]:
training13b_gt_snippets = {}
for element in training13b_data:
    training13b_gt_snippets[element['id']] = element['snippets']

In [36]:
# extract the text from the retrieved articles
retreived_articles_sampled_relevant_info = []
for object in retreived_articles_sampled_data:
    # we need for each article to convert pid from the id to link
    relevant_articles = []
    for article in object['details']:
        article['pid'] = convert_pid_to_link(article['pid'])
        relevant_articles.append(article)
    

    # randomize elements in articles
    random.shuffle(relevant_articles)


    relevant_element = {
        'qid': object['qid'],
        'question': object['question'],
        'ground_truth_documents_pid': [convert_pid_to_link(gt_pid) for gt_pid in object['ground_truth']], 
        'error_rate': object['error_rate'],
        'ground_truth_snippets': training13b_gt_snippets[object['qid']],
        'all_retreived_articles': relevant_articles
    }
    retreived_articles_sampled_relevant_info.append(relevant_element)

print(retreived_articles_sampled_relevant_info[0:3])

[{'qid': '67e6ce6e18b1e36f2e0000cf', 'question': 'How many primary genetic associations were identified through pQTL mapping within the Pharma Proteomics Project?', 'ground_truth_documents_pid': ['http://www.ncbi.nlm.nih.gov/pubmed/37794186'], 'error_rate': {'value': 1.0, 'details': '1 found out of 1'}, 'ground_truth_snippets': [{'beginSection': 'abstract', 'endSection': 'abstract', 'text': 'The Pharma Proteomics Project is a precompetitive biopharmaceutical consortium characterizing the plasma proteomic profiles of 54,219 UK Biobank participants. Here we provide a detailed summary of this initiative, including technical and biological validations, insights into proteomic disease signatures, and prediction modelling for various demographic and health indicators. We present comprehensive protein quantitative trait locus (pQTL) mapping of 2,923 proteins that identifies 14,287 primary genetic associations, of which 81% are previously undescribed, alongside ancestry-specific pQTL mapping i

In [37]:
# write this info to the json file
parsed_data_final = {
    'data': retreived_articles_sampled_relevant_info
}

data_serialized = json.dumps(parsed_data_final, indent = 4)

with open(parsed_data_path, 'w') as outfile:
    outfile.write(data_serialized)
